In [ ]:
#Code using inception transfer learning with Adam optimizer
import os
import zipfile
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import Model

# Using Transfer learning to build the model.
!wget --no-check-certificate \
https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5\
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file ='/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (100, 100, 3),
include_top = False,
weights = None)
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

from tensorflow.keras.optimizers import Adam
#Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
#Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
#Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
x = layers.Dense (8, activation='softmax')(x)
model = Model( pre_trained_model.input, x)
model.compile(optimizer = Adam(0.0001),
loss = 'categorical_crossentropy',
metrics = ['accuracy'])

from google.colab import drive
drive.mount("/content/drive")
local_zip = ’/content/drive/MyDrive/WEED/TRAIN-WEED.zip’
zip_ref = zipfile.ZipFile(local_zip, ’r’)
zip_ref.extractall(’/tmp/’)
zip_ref.close()
local_zip = ’/content/drive/MyDrive/WEED/VALIDATION-WEED.zip’
zip_ref = zipfile.ZipFile(local_zip, ’r’)
zip_ref.extractall(’/tmp/’)
zip_ref.close()

TRAINING_DIR = "/tmp/TRAIN-WEED/"
training_datagen = ImageDataGenerator(
rescale = 1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode=’nearest’)

VALIDATION_DIR = "/tmp/VALIDATION-WEED/"
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = training_datagen.flow_from_directory(
TRAINING_DIR,
target_size=(100,100),
class_mode=’categorical’,
batch_size=320
)

validation_generator = validation_datagen.flow_from_directory(
VALIDATION_DIR,
target_size=(100,100),
class_mode=’categorical’,
batch_size=200
)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get(’accuracy’)>0.98):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True
    callbacks = myCallback()
    history = model.fit(train_generator,
        epochs=150,
        steps_per_epoch=len(train_generator),
        validation_steps=len(validation_generator),
        verbose = 1,
        validation_data = validation_generator,
        callbacks=[callbacks]
        )


#To plot the training and validation accuracy/loss.
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()
